# Imports

In [1]:
!pip install accelerate -U
!pip install transformers
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 324.3/324.3 kB 6.3 MB/s eta 0:00:00
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.32.1
    Uninstalling accelerate-0.32.1:
      Successfully uninstalled accelerate-0.32.1
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 22.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.8 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.

In [4]:
from transformers import AutoModel, AutoTokenizer, Trainer, TrainingArguments, get_linear_schedule_with_warmup, BertModel, BertTokenizer, DataCollatorWithPadding
import numpy as np
import numpy
import pandas
from datasets import load_dataset
import torch
from torch.utils.data import Dataset, DataLoader
import nltk
from nltk.corpus import wordnet as wn
from nltk.corpus.reader.wordnet import Synset
from datasets import load_metric
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam
import spacy

import nltk

nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')
nltk.download('omw-1.4')

from nltk.corpus import wordnet as wn
from nltk.tokenize import word_tokenize ,sent_tokenize
from nltk.tag import pos_tag
from nltk.corpus.reader.wordnet import Lemma
import random
import re
from spacy.lang.en.stop_words import STOP_WORDS
from string import punctuation
import networkx as nx
from transformers import DistilBertTokenizer, DistilBertModel
from nltk.wsd import lesk
device = "cuda" if torch.cuda.is_available() else "cpu"

nlp = spacy.load('en_core_web_sm')

bert_tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')
bert_model = AutoModel.from_pretrained('bert-base-cased').to(device)

model_name = 'huawei-noah/TinyBERT_General_4L_312D'

distill_bert_tokenizer = DistilBertTokenizer.from_pretrained(model_name)
distill_bert_model = DistilBertModel.from_pretrained(model_name, output_attentions=True).to(device)

dataset = load_dataset("tommasobonomo/sem_augmented_fever_nli")

adversarial_fever_dataset = load_dataset("iperbole/adversarial_fever_nli")

out_classes = 3
fine_tune = True
epochs = 3
learning_rate = 1e-4
batch_size = 64

tokenizer = AutoTokenizer.from_pretrained(model_name)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
/usr/local/lib/python3.10/dist-packages/spacy/util.py:1740: UserWarning: [W111] Jupyter notebook detected: if using `prefer_gpu()` or `require_gpu()`, include it in the same cell right before `spacy.load()` to ensure that the model is loaded on the correct device. More information: http://spacy.io/usage/v3#jupyter-notebook-gpu
  warnings.warn(Warning

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/409 [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertTokenizer'. 
The class this function is called from is 'DistilBertTokenizer'.
You are using a model of type bert to instantiate a model of type distilbert. This is not supported for all configurations of models and can yield errors.


pytorch_model.bin:   0%|          | 0.00/62.7M [00:00<?, ?B/s]

Some weights of DistilBertModel were not initialized from the model checkpoint at huawei-noah/TinyBERT_General_4L_312D and are newly initialized: ['embeddings.LayerNorm.bias', 'embeddings.LayerNorm.weight', 'embeddings.position_embeddings.weight', 'embeddings.word_embeddings.weight', 'transformer.layer.0.attention.k_lin.bias', 'transformer.layer.0.attention.k_lin.weight', 'transformer.layer.0.attention.out_lin.bias', 'transformer.layer.0.attention.out_lin.weight', 'transformer.layer.0.attention.q_lin.bias', 'transformer.layer.0.attention.q_lin.weight', 'transformer.layer.0.attention.v_lin.bias', 'transformer.layer.0.attention.v_lin.weight', 'transformer.layer.0.ffn.lin1.bias', 'transformer.layer.0.ffn.lin1.weight', 'transformer.layer.0.ffn.lin2.bias', 'transformer.layer.0.ffn.lin2.weight', 'transformer.layer.0.output_layer_norm.bias', 'transformer.layer.0.output_layer_norm.weight', 'transformer.layer.0.sa_layer_norm.bias', 'transformer.layer.0.sa_layer_norm.weight', 'transformer.layer.

#Datasets

In [5]:
class NLIDataset(Dataset):
    def __init__(self, dataset, tokenizer, max_length=128, transform = False):
        self.dataset = dataset
        self.transform = transform

        self.tokenizer = tokenizer
        self.max_length = max_length

        self.premises = [word for word in self.dataset['premise']]
        self.hypothesis = [word for word in self.dataset['hypothesis']]
        self.labels = self.dataset['label']

    def __len__(self):
        return len(self.dataset['premise'])

    def __getitem__(self, idx):

        if self.transform:
          invert, transformed_sentence = transform_sentence(self.hypothesis[idx])
          inputs = self.tokenizer(
                    self.premises[idx],
                    transformed_sentence,
                    padding='max_length',
                    truncation=True,
                    max_length=self.max_length,
                    return_tensors="pt",
                  ).to(device)

          inputs = {key: val.squeeze(0) for key, val in inputs.items()}

          if invert:
                inputs['labels'] = torch.tensor(self.invert_labels(self.labels[idx]))
          else:
                inputs['labels'] = torch.tensor(self.convert_labels(self.labels[idx]))

        else:
          inputs = self.tokenizer(
                      self.premises[idx],
                      self.hypothesis[idx],
                      padding='max_length',
                      truncation=True,
                      max_length=self.max_length,
                      return_tensors="pt",
                    ).to(device)

          inputs = {key: val.squeeze(0) for key, val in inputs.items()}
          inputs['labels'] = torch.tensor(self.convert_labels(self.labels[idx]))
        return inputs

    def convert_labels(self, label):
        label_map = {"ENTAILMENT": 0, "NEUTRAL": 1, "CONTRADICTION": 2}
        return label_map[label]

    def invert_labels(self, label):
        label_map = {"ENTAILMENT": 2, "NEUTRAL": 1, "CONTRADICTION": 0}
        return label_map[label]

#Splitted Dataset

In [ ]:
class Splitted_NLI_Dataset(Dataset):
    def __init__(self, dataset, tokenizer, max_length=128, transform = False):
        self.dataset = dataset
        self.transform = transform

        self.tokenizer = tokenizer
        self.max_length = max_length

        self.premises = [word for word in self.dataset['premise']]
        self.hypothesis = [word for word in self.dataset['hypothesis']]
        self.labels = self.dataset['label']


    def __len__(self):
        return len(self.dataset['premise'])

    def __getitem__(self, idx):

        inputs1 = self.tokenizer(
                    self.premises[idx],
                    padding='max_length',
                    truncation=True,
                    max_length=self.max_length,
                    return_tensors="pt",
                  ).to(device)

        if self.transform:
          invert, transformed_sentence = transform_sentence(self.hypothesis[idx])
          inputs2 = self.tokenizer(
                    transformed_sentence,
                    padding='max_length',
                    truncation=True,
                    max_length=self.max_length,
                    return_tensors="pt",
                  ).to(device)

          inputs = {
              "input_ids" : [inputs1['input_ids'],inputs2['input_ids']],
              "attention_mask" : [inputs1['attention_mask'],inputs2['attention_mask']]
            }

          if invert:
                inputs['labels'] = torch.tensor(self.invert_labels(self.labels[idx]))
          else:
                inputs['labels'] = torch.tensor(self.convert_labels(self.labels[idx]))

        else:
          inputs2 = self.tokenizer(
                      self.hypothesis[idx],
                      padding='max_length',
                      truncation=True,
                      max_length=self.max_length,
                      return_tensors="pt",
                    ).to(device)


          inputs = {
            "input_ids" : [inputs1['input_ids'],inputs2['input_ids']],
            "attention_mask" : [inputs1['attention_mask'],inputs2['attention_mask']]
          }
          inputs['labels'] = torch.tensor(self.convert_labels(self.labels[idx]))
        return inputs

    def convert_labels(self, label):
        label_map = {"ENTAILMENT": 0, "NEUTRAL": 1, "CONTRADICTION": 2}
        return label_map[label]

    def invert_labels(self, label):
        label_map = {"ENTAILMENT": 2, "NEUTRAL": 1, "CONTRADICTION": 0}
        return label_map[label]

#Utilities

In [14]:
def compute_metrics(eval_pred):
    load_accuracy = load_metric("accuracy")
    load_f1 = load_metric("f1")
    load_recall = load_metric("recall")
    load_precision = load_metric("precision")

    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)

    accuracy = load_accuracy.compute(predictions=predictions, references=labels)["accuracy"]
    f1 = load_f1.compute(predictions=predictions, references=labels, average='weighted')["f1"]
    recall = load_recall.compute(predictions=predictions, references=labels, average='weighted')["recall"]
    precision = load_precision.compute(predictions=predictions, references=labels, average='weighted')["precision"]

    return {"accuracy": accuracy, "f1": f1, "recall": recall, "precision": precision}

def count_unique_words(sentence1, sentence2):
    """
    Counts the number of words in the second sentence that are not in the first sentence
    after removing punctuation and symbols.

    Args:
        sentence1 (str): The first sentence.
        sentence2 (str): The second sentence.

    Returns:
        int: The number of unique words in the second sentence that are not in the first sentence.
    """
    def preprocess(sentence):
        cleaned_sentence = re.sub(r'\W+', ' ', sentence.lower())
        return set(cleaned_sentence.split())

    words1 = preprocess(sentence1)
    words2 = preprocess(sentence2)

    unique_words_in_second = words2 - words1

    return len(unique_words_in_second)


#Transformer

In [7]:
class TransformerClassifier(nn.Module):
    def __init__(self, model_name, out_classes, fine_tune=True, dropout=0.3):
        super().__init__()
        self.out_classes = out_classes

        self.transformer = AutoModel.from_pretrained(model_name, output_hidden_states=True)

        for param in self.transformer.parameters():
            param.requires_grad = fine_tune

        self.dropout = nn.Dropout(dropout)

        self.classifier = nn.Sequential(
            nn.Linear(self.transformer.config.hidden_size, self.transformer.config.hidden_size // 2),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(self.transformer.config.hidden_size // 2, out_classes)
        )
        self.criterion = nn.CrossEntropyLoss()

    def forward(self, input_ids=None, attention_mask=None,  labels=None, compute_predictions=False, compute_loss=True):
        model_args = {
            "input_ids": input_ids,
            "attention_mask": attention_mask
        }
        outputs = self.transformer(**model_args)
        hidden_states = outputs.hidden_states[-4:]

        transformers_outputs_sum = torch.stack(hidden_states, dim=0).sum(dim=0)
        transformers_outputs_sum = self.dropout(transformers_outputs_sum)

        logits = self.classifier(self.dropout(transformers_outputs_sum)).mean(dim=1)

        output = {"logits": logits}
        if compute_predictions:
            predictions = logits.argmax(dim=-1)
            output["predictions"] = predictions

        if compute_loss and labels is not None:
            output["loss"] = self.criterion(logits, labels)

        return output

## Training

In [8]:
train_dataset = NLIDataset(dataset['train'], tokenizer)
validation_dataset = NLIDataset(dataset['validation'], tokenizer)
test_dataset = NLIDataset(dataset['test'], tokenizer)

adversarial_dataset = NLIDataset(adversarial_fever_dataset['test'], tokenizer)

simple_transformer_model = TransformerClassifier(model_name, out_classes,  fine_tune=fine_tune).to(device)

training_args = TrainingArguments(
    output_dir="training_dir",
    num_train_epochs = 1,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    warmup_steps=500,
    weight_decay=0.001,
    save_strategy="no",
    learning_rate=learning_rate,
    evaluation_strategy="epoch",
    logging_dir='./logs',
    logging_steps=10,
)

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
optimizer = Adam(simple_transformer_model.parameters(), lr=learning_rate)
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=training_args.warmup_steps,
    num_training_steps=training_args.num_train_epochs * len(train_dataset) // training_args.per_device_train_batch_size
)

trainer = Trainer(
    model=simple_transformer_model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=validation_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    optimizers=(optimizer, scheduler)
)

trainer.train()


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


<ipython-input-6-de6e32138060>:2: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  load_accuracy = load_metric("accuracy")


The repository for accuracy contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/accuracy.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


The repository for f1 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/f1.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


The repository for recall contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/recall.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


The repository for precision contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/precision.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


TrainOutput(global_step=799, training_loss=0.6095894054417617, metrics={'train_runtime': 195.9044, 'train_samples_per_second': 260.77, 'train_steps_per_second': 4.079, 'total_flos': 0.0, 'train_loss': 0.6095894054417617, 'epoch': 1.0})

In [9]:
trainer.evaluate()

{'eval_loss': 0.7630146741867065,
 'eval_accuracy': 0.6979895104895105,
 'eval_f1': 0.6912607632383422,
 'eval_recall': 0.6979895104895105,
 'eval_precision': 0.6995018940893121,
 'eval_runtime': 4.569,
 'eval_samples_per_second': 500.767,
 'eval_steps_per_second': 7.879,
 'epoch': 1.0}

In [10]:
test_results = trainer.predict(test_dataset)

print(test_results.metrics)

{'test_loss': 0.8148828148841858, 'test_accuracy': 0.6773065150852645, 'test_f1': 0.6693438414973893, 'test_recall': 0.6773065150852645, 'test_precision': 0.6765865109012813, 'test_runtime': 4.133, 'test_samples_per_second': 553.346, 'test_steps_per_second': 8.71}


In [11]:
test_results = trainer.predict(adversarial_dataset)

print(test_results.metrics)

{'test_loss': 1.3392868041992188, 'test_accuracy': 0.5103857566765578, 'test_f1': 0.511196077363068, 'test_recall': 0.5103857566765578, 'test_precision': 0.5161765089704886, 'test_runtime': 1.1763, 'test_samples_per_second': 286.497, 'test_steps_per_second': 5.101}


# Cross Encoder Classification

In [ ]:
class CrossEncoderClassifier(nn.Module):
    def __init__(self, model_name, out_classes, fine_tune=True, dropout=0.3, *args, **kwargs):
        super().__init__()
        self.out_classes = out_classes
        self.transformer = AutoModel.from_pretrained(model_name, output_hidden_states=True)

        if not fine_tune:
            for param in self.parameters():
                param.requires_grad = False

        self.dropout = nn.Dropout(dropout)
        self.classifier = nn.Sequential(
            nn.Linear(self.transformer.config.hidden_size * 2, self.transformer.config.hidden_size ),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(self.transformer.config.hidden_size , out_classes)
        )

        self.criterion = nn.CrossEntropyLoss()

    def forward(self, input_ids=None, attention_mask=None, token_type_ids=None, labels=None, compute_predictions=False, compute_loss=True):
        model_args1 = {"input_ids": input_ids[:,0,0].to(device), "attention_mask": attention_mask[:,0,0].to(device)}
        model_args2 = {"input_ids": input_ids[:,1,0].to(device), "attention_mask": attention_mask[:,1,0].to(device)}

        outputs1 = self.transformer(**model_args1)
        outputs2 = self.transformer(**model_args2)

        hidden_states1 = outputs1.hidden_states[-4:]
        transformers_outputs_sum1 = torch.stack(hidden_states1, dim=0).sum(dim=0).mean(dim=1)

        hidden_states2 = outputs2.hidden_states[-4:]
        transformers_outputs_sum2 = torch.stack(hidden_states2, dim=0).sum(dim=0).mean(dim=1)

        concatenated_output = torch.cat((transformers_outputs_sum1, transformers_outputs_sum2), dim=1)
        concatenated_output = self.dropout(concatenated_output)

        logits = self.classifier(concatenated_output)

        output = {"logits": logits}

        if compute_predictions:
            predictions = logits.argmax(dim=-1)
            output["predictions"] = predictions

        if compute_loss and labels is not None:
            output["loss"] = self.criterion(logits, labels)

        return output

splitted_train_dataset = Splitted_NLI_Dataset(dataset['train'], tokenizer)
splitted_validation_dataset = Splitted_NLI_Dataset(dataset['validation'], tokenizer)
splitted_test_dataset = Splitted_NLI_Dataset(dataset['test'], tokenizer)

splitted_adversarial_dataset = Splitted_NLI_Dataset(adversarial_fever_dataset['test'], tokenizer)

cross_encoder_model = CrossEncoderClassifier(model_name, out_classes, fine_tune=fine_tune).to(device)

In [ ]:
training_args = TrainingArguments(
    output_dir="training_dir",
    num_train_epochs=epochs,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    warmup_steps=500,
    weight_decay=0.001,
    save_strategy="no",
    learning_rate=learning_rate,
    eval_strategy="epoch",
    logging_dir='./logs',
    logging_steps=10,
)

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
optimizer = Adam(cross_encoder_model.parameters(), lr=learning_rate)
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=training_args.warmup_steps,
    num_training_steps=training_args.num_train_epochs * len(splitted_train_dataset) // training_args.per_device_train_batch_size
)

trainer = Trainer(
    model=cross_encoder_model,
    args=training_args,
    train_dataset=splitted_train_dataset,
    eval_dataset=splitted_validation_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    optimizers=(optimizer, scheduler)
)

trainer.train()

TrainOutput(global_step=2397, training_loss=0.5745174040334843, metrics={'train_runtime': 750.5959, 'train_samples_per_second': 204.182, 'train_steps_per_second': 3.193, 'total_flos': 0.0, 'train_loss': 0.5745174040334843, 'epoch': 3.0})

In [ ]:
trainer.evaluate()

{'eval_loss': 0.9976467490196228,
 'eval_accuracy': 0.6000874125874126,
 'eval_f1': 0.5859302935310076,
 'eval_recall': 0.6000874125874126,
 'eval_precision': 0.624121098579562,
 'eval_runtime': 7.0092,
 'eval_samples_per_second': 326.426,
 'eval_steps_per_second': 5.136,
 'epoch': 3.0}

In [ ]:
test_results = trainer.predict(splitted_test_dataset)

print(test_results.metrics)

{'test_loss': 1.0493978261947632, 'test_accuracy': 0.589855706165282, 'test_f1': 0.574933008480421, 'test_recall': 0.589855706165282, 'test_precision': 0.615299259604202, 'test_runtime': 6.1236, 'test_samples_per_second': 373.474, 'test_steps_per_second': 5.879}


In [ ]:
test_results = trainer.predict(splitted_adversarial_dataset)

print(test_results.metrics)

{'test_loss': 1.6153533458709717, 'test_accuracy': 0.4065281899109792, 'test_f1': 0.39461924379267554, 'test_recall': 0.4065281899109792, 'test_precision': 0.44812992084712994, 'test_runtime': 2.0344, 'test_samples_per_second': 165.65, 'test_steps_per_second': 2.949}


#Pipeline Adversarial generation



*   Select the most important words in the sentence using attention
*   Substitute the selected words with either hypernhim or hyponim randomly chosen between the best results
*   Split the sentence in subsentences and randomly reorder them



## Compute metrics

In [12]:
def get_sentence_embedding(sentence):
    inputs = bert_tokenizer(sentence, return_tensors='pt', truncation=True, padding=True, max_length=128).to(device)
    with torch.no_grad():
        outputs = bert_model(**inputs)
    cls_embedding = outputs.last_hidden_state[:, 0, :]
    return cls_embedding

def compute_similarity(sentence1, sentence2):
    embedding1 = get_sentence_embedding(sentence1)
    embedding2 = get_sentence_embedding(sentence2)
    cosine_similarity = F.cosine_similarity(embedding1, embedding2)
    cosine_distance = 1 - cosine_similarity

    return cosine_similarity, cosine_distance

## Sentence Transformation

In [13]:
def clean_token(token):
    """
    Cleans a given token by removing specific special tokens, non-alphanumeric characters,
    and short tokens (length <= 3).

    Args:
        token (str): The token to be cleaned.

    Returns:
        str: The cleaned token.
    """
    if isinstance(token, str):
        token = re.sub(r'[SEP]', '', token)
    if isinstance(token, str):
        token = re.sub(r'[CLS]', '', token)
    if isinstance(token, str):
        token = re.sub(r'[^a-zA-Z0-9\s]', '', token)
    if len(token) <= 3:
        token = ''
    return token

def textrank(text, top_n=5, window_size = 100):
    """
    Extracts the top N important words from a text using the TextRank algorithm.

    Args:
        text (str): The input text.
        top_n (int): The number of top words to extract.

    Returns:
        list: A list of the top N important words.
    """
    doc = nlp(text)
    stop_words = set(STOP_WORDS)
    pos_tag = ['NOUN', 'ADJ', 'VB', 'ADV']

    words = [token.text.lower() for token in doc if  token.text.lower() not in stop_words and token.text not in punctuation]

    graph = nx.Graph()
    graph.add_nodes_from(words)

    for i in range(len(words)):
        for j in range(i + 1, i + window_size):
            if j >= len(words):
                break
            graph.add_edge(words[i], words[j])

    scores = nx.pagerank(graph)
    ranked_words = sorted(scores, key=scores.get, reverse=True)
    return ranked_words[:top_n]

def extract_proper_nouns(text):
    """
    Extracts proper nouns

    Args:
        text (str): The input text.

    Returns:
        list: A list of tuples containing proper nouns and their character positions.
    """
    doc = nlp(text)
    nouns = [ent.text for ent in doc.ents if ent.label_ in ['PERSON', 'ORG', 'GPE', 'LOC']]

    proper_nouns=[]
    for noun in nouns:
        proper_nouns = proper_nouns + noun.split(" ")

    return proper_nouns

def find_important_words(sentence, device='cpu', max_num=5):
    """
    Identifies the most important words in a sentence based on attention scores from a BERT model.

    Args:
        sentence (str): The input sentence.
        device (str): device
        max_num (int): The maximum number of important words to return.

    Returns:
        list: A list of tuples containing the important words and their attention scores.
    """
    inputs = distill_bert_tokenizer(sentence, return_tensors='pt', truncation=True, padding=True).to(device)

    with torch.no_grad():
        outputs = distill_bert_model(**inputs)

    attentions = outputs.attentions
    mean_attentions = torch.stack(attentions).mean(dim=1).mean(dim=1).squeeze(0).mean(dim=0).mean(dim=0)
    tokens = distill_bert_tokenizer.convert_ids_to_tokens(inputs['input_ids'].squeeze(0))

    token_attention_pairs = []
    for token, attention in zip(tokens, mean_attentions):
        cleaned_token = clean_token(token)
        if cleaned_token:
            token_attention_pairs.append((cleaned_token, attention))

    token_attention_pairs.sort(key=lambda x: x[1], reverse=True)

    return token_attention_pairs[:max_num]

def replace_word(synset):
    if isinstance(synset, list):
        synset = random.choice(synset)

    if not isinstance(synset, Synset):
        return False, synset

    lemma = synset.lemmas()[0]
    pos = synset.pos()

    if pos in ['v']:
        antonyms = lemma.antonyms()
        if antonyms:
            return True, random.choice(antonyms).name()

        synonyms = [l.name() for l in synset.lemmas() if l.name() != lemma.name()]
        if synonyms:
            return False, random.choice(synonyms)

    elif pos == 'n':
        synonyms = [l.name() for l in synset.lemmas() if l.name() != lemma.name()]
        hypernyms = synset.hypernyms()
        hyponyms = synset.hyponyms()

        if synonyms:
            return False, random.choice(synonyms)
        elif hypernyms:
            return False, random.choice(hypernyms).lemmas()[0].name()
        elif hyponyms:
            return False, random.choice(hyponyms).lemmas()[0].name()

    return False, lemma.name()


def remove_nouns(key_words, nouns):
    """
    Removes proper nouns from a list of keywords.

    Args:
        key_words (list): The list of keywords.
        nouns (list): The list of proper nouns.

    Returns:
        list: The filtered list of keywords without proper nouns.
    """
    single_nouns = []
    for noun in nouns:
          single_nouns.append(noun.lower())

    return [word for word in key_words if word.lower() not in single_nouns]

def word_tokenizer(text):
    """
    Tokenizes a sentence and returns tokens with their start and end indices.

    Args:
        text (str): The input text.

    Returns:
        list: A list of tuples containing the token, start index, and end index.
    """
    doc = nlp(text)
    tokens_with_indices = [ token.text for i, token in enumerate(doc)]
    return tokens_with_indices

def transform_sentence(sentence):
    """
    Transforms a sentence by substituting important keywords with their synsets.

    Args:
        sentence (str): The input sentence.

    Returns:
        str: The transformed sentence with substituted keywords.
    """
    word_list = textrank(sentence)

    nouns = extract_proper_nouns(sentence)
    words = sentence.lower().split()
    word_list = remove_nouns(word_list, nouns)

    word_sense = []
    tokens = word_tokenizer(sentence)

    invert = False

    for word in word_list:
        synset = lesk(sentence.split(), word)
        invert = False

        if word in tokens:
          index = tokens.index(word)
          inv, new_word = replace_word(synset)
          if inv:
            invert = True
          if not new_word:
              new_word = word
          tokens[index] = new_word
    sentence = ' ' .join(tokens)

    return invert,  sentence


##Keywords extraction

In [15]:
import string
import re

transformed_dataset = []

for original_sentence in dataset['train']['hypothesis'][:100]:
    invert, new_sentence= transform_sentence(original_sentence)
    transformed_dataset.append(new_sentence)
    cosine_similarity, cosine_distance = compute_similarity(original_sentence, new_sentence)
    changed_words = count_unique_words(original_sentence, new_sentence)

    print(f"ORIGINAL : {original_sentence}")
    print(f"NEW : {new_sentence}")
    print(f"Changed words: {changed_words}")
    print(f"Cosine Similarity: {cosine_similarity}")
    print(f"Cosine Distance: {cosine_distance}")


ORIGINAL : Roman Atwood is a content creator.
NEW : Roman Atwood is a substance person .
Changed words: 2
Cosine Similarity: tensor([0.9635], device='cuda:0')
Cosine Distance: tensor([0.0365], device='cuda:0')
ORIGINAL : The Boston Celtics play their home games at TD Garden.
NEW : The Boston Celtics period_of_play their home game_equipment at TD Garden .
Changed words: 2
Cosine Similarity: tensor([0.9642], device='cuda:0')
Cosine Distance: tensor([0.0358], device='cuda:0')
ORIGINAL : There is a movie called The Hunger Games.
NEW : There is a moving-picture_show call_in The Hunger Games .
Changed words: 3
Cosine Similarity: tensor([0.9795], device='cuda:0')
Cosine Distance: tensor([0.0205], device='cuda:0')
ORIGINAL : Ryan Seacrest is a person.
NEW : Ryan Seacrest is a grammatical_category .
Changed words: 1
Cosine Similarity: tensor([0.9615], device='cuda:0')
Cosine Distance: tensor([0.0385], device='cuda:0')
ORIGINAL : Stranger than Fiction is a film.
NEW : Stranger than Fiction is a 

# Cross encoder transformer

In [ ]:
torch.cuda.empty_cache()

adv_train_dataset = NLIDataset(dataset['train'], tokenizer, transform = True)
adv_validation_dataset = NLIDataset(dataset['validation'], tokenizer, transform = True)
adv_test_dataset = NLIDataset(dataset['test'], tokenizer, transform = True)

adversarial_dataset = NLIDataset(adversarial_fever_dataset['test'], tokenizer)


In [ ]:
training_args = TrainingArguments(
    output_dir="training_dir",
    num_train_epochs=1,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    warmup_steps=500,
    weight_decay=0.001,
    save_strategy="no",
    learning_rate=2e-5,
    evaluation_strategy="epoch",
    logging_dir='./logs',
    logging_steps=10,
)

simple_transformer_model = TransformerClassifier(model_name, out_classes,  fine_tune=fine_tune).to(device)

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
optimizer = Adam(simple_transformer_model.parameters(), lr=learning_rate)
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=training_args.warmup_steps,
    num_training_steps=training_args.num_train_epochs * len(adv_train_dataset) // training_args.per_device_train_batch_size
)


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
trainer = Trainer(
    model=simple_transformer_model,
    args=training_args,
    train_dataset=adv_train_dataset,
    eval_dataset=adv_validation_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    optimizers=(optimizer, scheduler)
)

trainer.train()

<ipython-input-4-de6e32138060>:2: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  load_accuracy = load_metric("accuracy")


The repository for accuracy contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/accuracy.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


The repository for f1 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/f1.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


The repository for recall contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/recall.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


The repository for precision contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/precision.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


TrainOutput(global_step=799, training_loss=0.635892332719175, metrics={'train_runtime': 3122.834, 'train_samples_per_second': 16.359, 'train_steps_per_second': 0.256, 'total_flos': 0.0, 'train_loss': 0.635892332719175, 'epoch': 1.0})

In [ ]:
trainer.evaluate()

{'eval_loss': 0.8269271850585938,
 'eval_accuracy': 0.6678321678321678,
 'eval_f1': 0.6578189364726194,
 'eval_recall': 0.6678321678321678,
 'eval_precision': 0.676694928119671,
 'eval_runtime': 58.3572,
 'eval_samples_per_second': 39.207,
 'eval_steps_per_second': 0.617,
 'epoch': 1.0}

In [ ]:
test_results = trainer.predict(adv_test_dataset)

print(test_results.metrics)

{'test_loss': 0.8814402222633362, 'test_accuracy': 0.6484477481416703, 'test_f1': 0.63723744788867, 'test_recall': 0.6484477481416703, 'test_precision': 0.6577373140690079, 'test_runtime': 58.9904, 'test_samples_per_second': 38.769, 'test_steps_per_second': 0.61}


In [ ]:
test_results = trainer.predict(adversarial_dataset)

print(test_results.metrics)

{'test_loss': 1.318281888961792, 'test_accuracy': 0.516320474777448, 'test_f1': 0.5193618273006008, 'test_recall': 0.516320474777448, 'test_precision': 0.5508681572160129, 'test_runtime': 2.4909, 'test_samples_per_second': 135.292, 'test_steps_per_second': 2.409}


#Bi-encoder transformer

In [ ]:
torch.cuda.empty_cache()

cross_train_dataset = Splitted_NLI_Dataset(dataset['train'][:20000], tokenizer, transform = True)
cross_validation_dataset = Splitted_NLI_Dataset(dataset['validation'], tokenizer, transform = True)
cross_test_dataset = Splitted_NLI_Dataset(dataset['test'], tokenizer, transform = True)

cross_adversarial_dataset = Splitted_NLI_Dataset(adversarial_fever_dataset['test'], tokenizer)


training_args = TrainingArguments(
    output_dir="training_dir",
    num_train_epochs=2,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    warmup_steps=500,
    weight_decay=0.001,
    save_strategy="no",
    learning_rate=1e-5,
    evaluation_strategy="epoch",
    logging_dir='./logs',
    logging_steps=10,
)

cross_transformer_model = CrossEncoderClassifier(model_name, out_classes, fine_tune=fine_tune).to(device)

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
optimizer = Adam(cross_transformer_model.parameters(), lr=learning_rate)
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=training_args.warmup_steps,
    num_training_steps=training_args.num_train_epochs * len(cross_train_dataset) // training_args.per_device_train_batch_size
)

trainer = Trainer(
    model=cross_transformer_model,
    args=training_args,
    train_dataset = cross_train_dataset,
    eval_dataset = cross_validation_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    optimizers=(optimizer, scheduler)
)

trainer.train()

In [ ]:
trainer.evaluate()

In [ ]:
test_results = trainer.predict(cross_test_dataset)

print(test_results.metrics)

In [ ]:
test_results = trainer.predict(cross_adversarial_dataset)

print(test_results.metrics)